<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">
           <a href="https://bokeh.org/" target="_blank">
           <img
               src="assets/bokeh-transparent.png"
               style="width:50px"
           >
           </a>
       </td>
       <td style="border:none">
           <h1>Bokeh Tutorial</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><a href="TOC.ipynb" target="_blank">Table of contents</a><br><h2>09 More plot types</h2></div>

In [ ]:
# load tutorial data
from tutorial_data import data

In [ ]:
# activate notebook output
from bokeh.io import output_notebook

output_notebook()

This chapter introduces **two additional categories of plot types**:
- [Map plots](#Map-plots)
- [Wedge plots](#Wedge-plots) (pie and donut charts)

### Map plots

Bokeh has several functionalities built-in that you can use to **visualize geographical
data on maps**.

#### Tile providers

Bokeh works with multiple tile providers that support Web Mercator projection.

Bokeh uses the [xyzservices](https://xyzservices.readthedocs.org/) library to manage
map tile providers.

This way, you can use popular map designs from providers such as OpenStreetMap, CartoDB,
Stamen, and Esri.

In order to use tiles in a plot, you first need to configure your plot. This is done
with the `figure()` function:
* The `x_range` and `y_range` parameters need to be defined in Web Mercator coordinates.
* The `x_axis_type` and `y_axis_type` parameters need to be set to `mercator`.

After configuring your plot, use the `add_tile()` function to add the map tiles to a plot.
This function requires the name of a tile source as a parameter.

This works similarly to adding a line render with `line()`, or adding bars with
`vbar()`:

In [ ]:
import xyzservices.providers as xyz

from bokeh.plotting import figure, show

# range bounds supplied in Web Mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000), x_axis_type="mercator", y_axis_type="mercator")

# 🔁 try uncommenting different lines to see different tile providers
p.add_tile("CartoDB Positron", retina=True)
# p.add_tile("Esri World Imagery", retina=True)
# p.add_tile("OSM", retina=True)
# p.add_tile("Stamen Terrain", retina=True)
# p.add_tile("Stamen.Watercolor", retina=True)

show(p)

To navigate around the map, use the standard Bokeh tools, such as the box zoom or wheel
zoom!

For more information on adding tiles to a plot, see the following resources:
* [Tile provider maps](https://docs.bokeh.org/en/latest/docs/user_guide/topics/geo.html#tile-provider-maps)
in the user guide 
* [Documentation for `add_tile`](https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html#bokeh.plotting.figure.add_tile)
  in the reference guide.
* the [tile_demo](https://docs.bokeh.org/en/latest/docs/examples/topics/geo/tile_demo.html)
  example in the Bokeh gallery


#### Google maps

Bokeh also supports Google Maps tiles. To use Google Maps tiles, you need to provide
an API key.

This makes it difficult to demonstrate Google Maps for the purpose of this tutorial. See
[Google Maps](https://docs.bokeh.org/en/latest/docs/user_guide/topics/geo.html#google-maps)
in the user guide for more information.

#### GeoJSONDataSource

To process data with geo information, Bokeh includes the GeoJSONDataSource. The
GeoJSONDataSource works similarly to the [ColumnDataSource](06_data_sources.ipynb).

You can use [GeoJSON](https://geojson.org/) data such as points, lines, and polygons
(called Patches in Bokeh) together with other data from a ColumnDataSource.



To demonstrate this, let's create a small GeoJSON. It contains the location of the UN
headquarters as a single point (40.749444, -73.968056, according to [Wikipedia](https://geohack.toolforge.org/geohack.php?pagename=Headquarters_of_the_United_Nations&params=40_44_58_N_73_58_5_W_type:landmark)):

In [ ]:
from shapely.geometry import Point
from pyproj import CRS, Transformer
import geojson

# Create a Point geometry for the UN headquarters (located at 40.748817, -73.968285)
un_location = Point(-73.968285, 40.748817)

# Create a GeoJSON feature using the Point geometry
un_feature = geojson.Feature(geometry=un_location, properties={"name": "United Nations headquarters"})

# Create a GeoJSON FeatureCollection containing the feature
un_feature_collection = geojson.FeatureCollection([un_feature])

# Print the FeatureCollection as GeoJSON string
geojson.dumps(un_feature_collection)

Bokeh uses the Web Mercator projection for map plots. Therefore, you need to convert
the GeoJSON to the Web Mercator projection. This example uses geopandas
to do this:

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame.from_features(un_feature_collection, crs="EPSG:4326")
un_web_mercator = gdf.to_crs(epsg=3857).to_json()
un_web_mercator

Now, we can use the GeoJSON to create a GeoJSONDataSource:

In [ ]:
from bokeh.models import GeoJSONDataSource

geo_source = GeoJSONDataSource(geojson=un_web_mercator)

When converting the GeoJSON to a GeoJSONDataSource, Bokeh automatically detects the
geometry. The GeoJSONDataSource contains two special columns:

* `x` and `y` for points
* `xs` and `ys` for lines and polygons

Bokeh creates those columns for you, based on the GeoJSON geometry.

You can use the GeoJSONDataSource to draw the point on a plot. This works just like
drawing glyphs from a ColumnDataSource:

In [ ]:
# set up a figure covering North America
p = figure(
    x_range=(-14000000, -6000000),  # x range bounds supplied in Web Mercator coordinates
    y_range=(2000000, 4000000),  # y range bounds supplied in Web Mercator coordinates
    x_axis_type="mercator",  # set the x-axis to use Web Mercator coordinates
    y_axis_type="mercator",  # set the y-axis to use Web Mercator coordinates
    tools="pan, wheel_zoom, reset",  # configure the toolbar for zooming and panning
    active_scroll="wheel_zoom",
)

# add a circle glyph to represent the UN headquarters
p.scatter(
    x="x",  # use the "x" column from the GeoJSONDataSource
    y="y",  # use the "y" column from the GeoJSONDataSource
    fill_color="#5B92E5",
    line_color="white",
    size=25,
    fill_alpha=0.8,
    source=geo_source,  # use the GeoJSONDataSource as the source
)

# add a tile source for the base map
p.add_tile("Esri World Imagery", retina=True)

show(p)

Use the pan and wheel zoom tools to navigate around the map. You can see where
exactly the UN headquarters is located!

#### GeoJSON map example

One of the plots in the demo dashboard is a map of the US states. This map visualizes
the number of routes that originate in each state.

The map visualization consists only of polygons and does not use a tile provider.

This map visualization uses two data sets:
1. A GeoJSON file with the shapes of all US states. Alaska and Hawaii are moved from
their actual location to make visualization easier:

In [ ]:
states_gdf = gpd.read_file("../data/us-states.geojson")
states_gdf.plot()  # use geopandas to plot the state shapes

2. A pandas DataFrame with the number of routes beginning and ending in each state:

In [ ]:
# read the pre-processed data frame from the demo data set
data.get_states_routes_df().head(2)

To create the basis for the GeoJSONDataSource in this example, first join the state
shapes with the number of routes:

In [ ]:
states_gdf = states_gdf.join(data.get_states_routes_df(), on=states_gdf["Name"])
states_gdf.head(2)

The next code cell contains the code to create this visualization.

You'll recognize several elements from the previous chapters. For example:
* Tooltips
* Configuring plot tools
* Configuring visual elements of the plot
* Using a ColorMapper to map values to colors from a palette
* Adding a color bar as a visual guide

In [ ]:
import geopandas as gpd

from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, NumeralTickFormatter
from bokeh.palettes import Cividis
from bokeh.transform import linear_cmap

# read the GeoJSON file containing the state shapes
states_gdf = gpd.read_file("../data/us-states.geojson")
# read the pre-processed data frame from the demo data set and join it to the state shapes
states_gdf = states_gdf.join(data.get_states_routes_df(), on=states_gdf["Name"])
# create the GeoJSONDataSource
geo_source = GeoJSONDataSource(geojson=states_gdf.to_json())

# set up the tooltips
TOOLTIPS = [
    ("State", "@Name"),
    ("# of routes departing from here", "@origin{(0,0)}"),
]

# set up the figure
map_plot = figure(
    height=200,  # set a width and height to define the aspect ratio
    width=300,
    sizing_mode="scale_width",
    tooltips=TOOLTIPS,
    title="Number of routes with a state as its origin (all domestic carriers)",
    x_axis_location=None,  # deactivate x-axis
    y_axis_location=None,  # deactivate y-axis
    toolbar_location=None,  # deactivate toolbar
)
map_plot.grid.grid_line_color = None  # make grid lines invisible

# draw the state polygons
us = map_plot.patches(  # use the patches method to draw the polygons of all states
    xs="xs",
    ys="ys",
    fill_color=linear_cmap(field_name="origin", palette=Cividis[256], low=states_gdf["origin"].min(), high=states_gdf["origin"].max()),  # color the states by mapping the number of routes to color values from a palette
    source=geo_source,
    line_color="darkgrey",
    line_width=1,
)

# add color bar
color_bar = us.construct_color_bar(formatter=NumeralTickFormatter(format="0,0"), height=10)
map_plot.add_layout(obj=color_bar, place="below")

show(map_plot)

### Wedge plots

You can use Bokeh's ``wedge`` glyph method to create donut and pie charts.

#### Pie charts

Let's create a pie chart representing the shares of pizza toppings ordered at a pizza
place in one day

First, create a DataFrame with the number of orders for each topping:

In [ ]:
import pandas as pd

pizza_df = pd.DataFrame(
    {
        "topping": ["Pepperoni", "Cheese", "Mixed Veggies", "Bacon"],  # the four available toppings
        "orders": [221, 212, 152, 72],  # the number of orders for each topping
        "color": ["red", "darkorange", "darkgreen", "hotpink"],  # define colors for each topping
    }
)
pizza_df

Next, calculate the angle for each wedge (in radians):

In [ ]:
from math import pi

pizza_df["angle"] = pizza_df["orders"] / pizza_df["orders"].sum() * 2 * pi
pizza_df

Then, convert the DataFrame to a ColumnDataSource and visualize the data:

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.transform import cumsum

pizza_data = ColumnDataSource(pizza_df)  # create a ColumnDataSource from the DataFrame

# set up the figure
pizza_plot = figure(
    height=350,
    toolbar_location=None,  # deactivate the toolbar
    tooltips="@topping: @orders orders",  # configure tooltips to use the "topping" and "orders" columns
    x_range=(-0.5, 1.0),
    background_fill_color="#1d1d1d",  # set the background color to a dark gray
)

# define the pie chart
pizza_plot.wedge(
    x=0,  # x coordinate of the center of the pie chart
    y=1,  # y coordinate of the center of the pie chart
    radius=0.4,  # radius of the pie chart
    start_angle=cumsum("angle", include_zero=True),
    end_angle=cumsum("angle"),
    line_color="#94541e",  # approximate the color of a pizza crust
    line_width=6,
    fill_color="color",  # color each wedge based on the "color" column
    legend_field="topping",  # use the "topping" column as legend entries
    source=pizza_data,  # use the ColumnDataSource as the source
)
pizza_plot.axis.visible = False  # deactivate the axes
pizza_plot.grid.grid_line_color = None  # deactivate the grid lines
pizza_plot.legend.background_fill_alpha = 0  # make the legend background transparent

show(pizza_plot)

For more information on pie charts, see [Pie chart](https://docs.bokeh.org/en/latest/docs/user_guide/topics/pie.html#pie-chart)
in the user guide.

#### Donut charts

Donut charts are like pie charts but with a hole in the middle. They work very similarly
to pie charts. The only difference is that they use the `annularwedge` method instead of
the `wedge` method.

The following example is similar to the pie chart above. It uses different donut
flavors instead of pizza toppings:

In [ ]:
import pandas as pd
from math import pi
from bokeh.io import curdoc

donut_df = pd.DataFrame(
    {
        "topping": ["Chocolate frosting", "Powdered sugar", "Strawberry frosting", "Purple glaze"],  # the four available toppings
        "orders": [723, 631, 592, 319],  # the number of orders for each topping
        "color": ["saddlebrown", "cornsilk", "crimson", "purple"],  # define colors for each topping
    }
)

donut_df["angle"] = donut_df["orders"] / donut_df["orders"].sum() * 2 * pi

from bokeh.models import ColumnDataSource
from bokeh.transform import cumsum

donut_data = ColumnDataSource(donut_df)  # create a ColumnDataSource from the DataFrame

# set up the figure
donut_plot = figure(
    height=350,
    toolbar_location=None,  # deactivate the toolbar
    tooltips="@topping: @orders orders",  # configure tooltips to use the "topping" and "orders" columns
    x_range=(-0.5, 1.0),
    background_fill_color="#1d1d1d",  # set the background color to a dark gray
)

# define the donut chart
donut_plot.annular_wedge(
    x=0,  # x coordinate of the center of the donut chart
    y=1,  # y coordinate of the center of the donut chart
    outer_radius=0.4,  # outer radius of the donut chart
    inner_radius=0.2,  # inner radius of the donut chart
    start_angle=cumsum("angle", include_zero=True),
    end_angle=cumsum("angle"),
    line_color="#e89045",
    line_width=6,
    fill_color="color",  # color each annular wedge based on the "color" column
    legend_field="topping",  # use the "topping" column as legend entries
    source=donut_data,  # use the ColumnDataSource as the source
)
donut_plot.axis.visible = False  # deactivate the axes
donut_plot.grid.grid_line_color = None  # deactivate the grid lines
donut_plot.legend.background_fill_alpha = 0  # make the legend background transparent

show(donut_plot)

For more information on donut charts, see [Donut chart](https://docs.bokeh.org/en/latest/docs/user_guide/topics/pie.html#donut-chart)
in the user guide.

#### Donut chart example

One of the plots in the demo dashboard is a donut chart. This chart visualizes the top
10 carriers by passengers, freight, or mail.

You will learn how to build the tabs for switching between passengers, freight, and
mail in a later chapter. For now, let's focus on building a donut chart displaying
the top ten domestic carriers by passengers.

First, create a DataFrame with the top carriers by number of passengers. This data is
part of the pre-processed demo data set:

In [ ]:
carrier_df = data.get_top_carriers_by_metrics("passengers")
carrier_df.head()

This DataFrame contains the carrier names and the number of passengers. For convenience,
it also contains pre-computed angles and colors for each of the wedges.

The following code cell contains the code to create the donut chart:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.transform import cumsum

# configure tooltips
TOOLTIPS = [
    ("Carrier", "@unique_carrier_name"),
    ("Passengers", "@passengers{(0,0)}"),
]

# create a ColumnDataSource from the DataFrame
passengers_data = ColumnDataSource(carrier_df)

# set up the figure
annular_plot = figure(
    height=350,
    toolbar_location=None,
    outline_line_color=None,
    name="region",
    x_range=(-0.66, 1),
    title="Top ten carriers by passengers (domestic)",
    tooltips=TOOLTIPS,
)

# define the annular chart
annular_plot.annular_wedge(
    x=0,
    y=1,
    inner_radius=0.2,
    outer_radius=0.4,
    start_angle=cumsum("angle", include_zero=True),
    end_angle=cumsum("angle"),
    line_color="white",
    fill_color="color",
    legend_field="unique_carrier_name",
    source=passengers_data,
)

annular_plot.axis.visible = False
annular_plot.grid.grid_line_color = None

show(annular_plot)

# Next section

<a href="10_layouts.ipynb" target="_blank">
    <img src="assets/arrow.svg" alt="Next section" width="100" align="right">
</a>

In the [next chapter](10_layouts.ipynb), you'll learn how to combine multiple plots
into layouts.